#14_CNN_LeNet5 & MNIST

###ML 라이브러리: Pytorch(GPU)

In [ ]:
import torch
import torchvision.datasets as data
import torchvision.transforms as transforms
import torch.nn.init

GPU 사용을 명시적으로 보여주기 & Random Seed 고정하기

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

학습 파라미터 설정

In [ ]:
learning_rate = 0.9
training_epochs = 50
batch_size = 256

##1. 데이터셋 불러오기

TorchVision의 MNIST 영상이 28x28로 가공되어 있으므로, LeNet5의 Original 입력 사이즈 32x32로 맞춰야 한다

In [ ]:
transforms = transforms.Compose([transforms.Resize((32, 32)), transforms.ToTensor()])

In [ ]:
mnist_train = data.MNIST(root = 'MNIST_data/', train = True, transform = transforms, download = True)

mnist_test = data.MNIST(root = 'MNIST_data/', train = False, transform = transforms, download = True)

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


1. train_loader

In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=mnist_train, batch_size=batch_size, shuffle=True, drop_last=True)
#dataset = mnist_train: 처리할 데이터
#batch_size = batch_size: batch_size만큼씩 데이터를 잘라러 준다
#shuffle = True: 데이터를 섞는다
#drop_last: 모든 데이터 개수 % batch_size > 0 일 때 나머지 데이터를 버린다

2. test_loader

In [ ]:
test_loader = torch.utils.data.DataLoader(dataset=mnist_test, batch_size=batch_size, shuffle=True, drop_last=True)

##5. 모델 추정

###모델 정의: LeNet5

In [ ]:
class LeNet5(torch.nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()

        self.c1 = torch.nn.Conv2d(1, 6, kernel_size = 5, stride = 1, padding = 0)
        self.x1 = torch.nn.Tanh()
        self.s2 = torch.nn.AvgPool2d(kernel_size = 2, stride = 2)

        self.c3 = torch.nn.Conv2d(6, 16, kernel_size = 5, stride = 1, padding = 0)
        self.x3 = torch.nn.Tanh()
        self.s4 = torch.nn.AvgPool2d(kernel_size = 2, stride = 2)

        self.fl = torch.nn.Faltten()

        self.f5 = torch.nn.Linear(16 * 5 * 5, 120, bias = True)
        self.x5 = torch.nn.Tanh()

        self.f6 = torch.nn.Linear(120, 84, bias = True)
        self.x6 = torch.nn.Tanh()

        self.ouput = torch.nn.Linear(84, 10, bias = True)
    

    def forward(self, x):
        out = self.c1(x)
        out = self.x1(out)
        out = self.s2(out)

        out = self.c3(out)
        out = self.x3(out)
        out = self.s4(out)

        out = out.view(out.size(0), -1) # batch x 1 x 1 x 120

        out = self.f5(out)
        out = self.x5(out)

        out = self.f6(out)    
        out = self.x6(out)

        out = self.ouput(out)

        return out

LeNet with Sigmoid

In [ ]:
class LeNet5_Sigmoid(torch.nn.Module):
    def __init__(self):
        super(LeNet5_Sigmoid, self).__init__()

        self.c1 = torch.nn.Conv2d(1, 6, kernel_size = 5, stride = 1, padding = 0)
        self.x1 = torch.nn.Sigmoid()
        self.s2 = torch.nn.AvgPool2d(kernel_size = 2, stride = 2)

        self.c3 = torch.nn.Conv2d(6, 16, kernel_size = 5, stride = 1, padding = 0)
        self.x3 = torch.nn.Sigmoid()
        self.s4 = torch.nn.AvgPool2d(kernel_size = 2, stride = 2)

        # self.c5 = torch.nn.Conv2d(16, 120, kernel_size = 5, stride = 1, padding = 0)
        self.f5 = torch.nn.Linear(16 * 5 * 5, 120, bias = True)
        self.x5 = torch.nn.Sigmoid()

        self.f6 = torch.nn.Linear(120, 84, bias = True)
        self.x6 = torch.nn.Sigmoid()

        self.ouput = torch.nn.Linear(84, 10, bias = True)
    

    def forward(self, x):
        out = self.c1(x)
        out = self.x1(out)
        out = self.s2(out)

        out = self.c3(out)
        out = self.x3(out)
        out = self.s4(out)

        out = out.view(out.size(0), -1) # batch x 1 x 1 x 120

        out = self.f5(out)
        out = self.x5(out)

        out = self.f6(out)    
        out = self.x6(out)

        out = self.ouput(out)

        return out

모델 호출

In [ ]:
model = LeNet5_Sigmoid().to(device)

###모델 학습

Cross Entropy Loss & SGD

In [ ]:
loss = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

모델 학습

In [ ]:
total_batch = len(train_loader)

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in train_loader:
        X = X.to(device)
        Y = Y.to(device)

        # Gradient Initialization
        optimizer.zero_grad()

        # Forward 계산
        hypothesis = model(X)

        # Cost : Error 계산
        cost = loss(hypothesis, Y)

        # Backpropagation
        cost.backward()

        # Cost로 가중치(W, b) 갱신
        optimizer.step()

        # 평균 Error 계산
        avg_cost += cost / total_batch
    
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

[Epoch:    1] cost = 2.31523395
[Epoch:    2] cost = 2.30319571
[Epoch:    3] cost = 2.30281234
[Epoch:    4] cost = 2.30269861
[Epoch:    5] cost = 2.30237031
[Epoch:    6] cost = 2.30224419
[Epoch:    7] cost = 2.3020699
[Epoch:    8] cost = 2.30204368
[Epoch:    9] cost = 2.30170465
[Epoch:   10] cost = 2.30132294
[Epoch:   11] cost = 2.30053544
[Epoch:   12] cost = 2.28415656
[Epoch:   13] cost = 1.88537371
[Epoch:   14] cost = 0.866533995
[Epoch:   15] cost = 0.452311903
[Epoch:   16] cost = 0.299176604
[Epoch:   17] cost = 0.227376193
[Epoch:   18] cost = 0.179459125
[Epoch:   19] cost = 0.15016681
[Epoch:   20] cost = 0.129572734
[Epoch:   21] cost = 0.114021726
[Epoch:   22] cost = 0.101893373
[Epoch:   23] cost = 0.0907776579
[Epoch:   24] cost = 0.0865685716
[Epoch:   25] cost = 0.0785702914
[Epoch:   26] cost = 0.0739649162
[Epoch:   27] cost = 0.0688718036
[Epoch:   28] cost = 0.0641224831
[Epoch:   29] cost = 0.0612614118
[Epoch:   30] cost = 0.0575715564
[Epoch:   31] cos

##6. 결과 분석

Accuracy

In [ ]:
accuracy = 0
total_batch = len(test_loader)

with torch.no_grad():
    model.eval()

    for X, Y in test_loader:
        X = X.to(device)
        Y = Y.to(device)

        prediction = model(X)
        correct_prediction = torch.argmax(prediction, 1) == Y
        accuracy += correct_prediction.float().mean()

print('Accuracy:', accuracy.item()/total_batch)

Accuracy: 0.9881810897435898
